In [1]:
from ekonlpy.sentiment import MPCK
from tqdm import tqdm
from datetime import datetime, timedelta

import MeCab, csv
import pandas as pd
import numpy as np

mpck = MPCK()

# **1. N그램화**

In [3]:
# 2008년 11월 ~ 2012년 데이터
news2 = pd.read_csv('naver_2008nov_2012.csv', header=0, sep=',')

news2 = news2['text']
news2 = list(news2)

news_content2 = []
news_error2 = []
for i, article in tqdm(enumerate(news2)):
    tmp=[]
    try: 
        tokens = mpck.tokenize(article)
        ngrams = mpck.ngramize(tokens)
        tmp.append(','.join(tokens))
        tmp.append(','.join(ngrams))
    except:
        news_error2.append(i)
    news_content2.append(tmp)
    
pd.DataFrame(news_content2).to_csv('Ngramize_2008_2012.csv',header=0, sep=',')

56455it [19:55, 47.22it/s]


In [9]:
# 2013년 ~ 2017년 데이터

news3= pd.read_csv('naver_2013_2017.csv', header=0, sep=',')

news3 = news3['text']
news3 = list(news3)

news_content3 = []
news_error3 = []
for i, article in tqdm(enumerate(news3)):
    tmp=[]
    try: 
        tokens = mpck.tokenize(article)
        ngrams = mpck.ngramize(tokens)
        tmp.append(','.join(tokens))
        tmp.append(','.join(ngrams))
    except:
        news_error3.append(i)
    news_content3.append(tmp)
    
pd.DataFrame(news_content3).to_csv('Ngramize_2013_2017.csv',header=0, sep=',')

75302it [28:46, 43.60it/s]


In [11]:
pd.read_csv('Ngramize_2008_2012.csv', header=0, sep=','), pd.read_csv('Ngramize_2013_2017.csv', header=0, sep=',')

(           0  \
 0          1   
 1          2   
 2          3   
 3          4   
 4          5   
 ...      ...   
 56449  56450   
 56450  56451   
 56451  56452   
 56452  56453   
 56453  56454   
 
       뉴욕/NNG,특파원/NNG,현지/NNG,뉴욕/NNG,주식시장/NNG,보합권/NNG,등락/NNG,거듭/MAG,하/VV,전역/NNG,제조업/NNG,경기/NNG,가늠/NNG,하/XSV,ism/NNG,ism/NNG,지수/NNG,최저/NNG,추락/NNG,우려/NNG,고조/NNG,되/XSV,리보/NNG,리보금리/NNG,사태/NNG,최저/NNG,떨어/VV,신용경색/NNG,지속/NNG,완화/NNG,되/XSV,소식/NNG,긍정적/VAX,요인/NNG,작용/NNG,하/XSV,금융위기/NNG,최악/NNG,벗어나/VV,기대/NNG,크/VV,또/MAG,커뮤니케이션/NNG,주가/NNG,비아/NNG,호평/NNG,동반/NNG,상승/NNG,하/XSV,투자심리/NNG,보탬/VV,주/VV,블루칩/NNG,중심/NNG,다우존스산업평균지수/NNG,거래일/NNG,대비/NNG,포인트/NNG,상승/NNG,하/VV,중심/NNG,나스닥/NNG,포인트/NNG,오르/VV,기록/NNG,중/NNG,대형주/NNG,중심/NNG,s&p500/NNG,포인트/NNG,전진/NNG,하/VV,국제/NNG,유가/NNG,내림/NNG,서부텍사스산원유/NNG,인도분/NNG,대비/NNG,달러/NNG,떨어/VV,배럴/NNG,달러/NNG,기록/NNG,하/XSV,주인/NNG,커뮤니케이션/NNG,주식/NNG,밸류에이션/NNG,괜찮/VA,경기/NNG,침체국면/NNG,안전자산/NNG,되/VV,있/VV,비아/NNG,긍정적/VAX,평가/NNG,호재/NNG,작용/NNG,오르/VV,서킷/NNG,시티/NNG,경영난/NNG,타개/NNG,위하/VV,전역/NNG,점포/NNG,중/NNG,문/

# **2. 금리데이터**

In [2]:
call_rate = pd.read_csv('CallRate_1.csv', header=None)
call_rate.columns = ['date', 'call_rate']
call_rate['date'] = call_rate['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) #str형식의 date를 date형식으로 변환
call_rate['call_rate'] = call_rate['call_rate'].fillna(method = 'ffill') #결측치는 가장 가까운 전날의 금리로 채운다
call_rate

,date,call_rate
0,2004-11-01,3.53
1,2004-11-02,3.53
2,2004-11-03,3.53
3,2004-11-04,3.53
4,2004-11-05,3.52
...,...,...
4804,2017-12-27,1.51
4805,2017-12-28,1.55
4806,2017-12-29,1.58
4807,2017-12-30,1.58


# **3. 의사록 데이터**

In [3]:
minutes_data = pd.read_csv('minutes_handle_ir.csv', header=0, index_col=[0])
minutes_ngrams = list(minutes_data['ngram'])
minutes_data['label'] = minutes_data['change'].apply(lambda x: 1 if x > 0.03 else -1 if x < -0.03 else 0 ) #threshold 적용하면 48개 남음
minutes_data['token_ngram'] = 0

for i, minute_ngram in enumerate(minutes_ngrams):
    minute_ngram = minute_ngram.replace("[", "")
    minute_ngram = minute_ngram.replace("]", "")
    minutes_data['token_ngram'][i] = minute_ngram

minutes = minutes_data[['token_ngram', 'label']]
minutes

minutes1 = minutes_data[['date', 'token_ngram', 'label']]

C:\Users\student\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\student\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [286]:
minutes1.to_csv('minutes_label.csv', encoding='utf-8')

# **4. 데이터 합치기**

In [4]:
news = pd.read_csv('news_re.csv', header=0, index_col=[0])
news_ngrams = list(news['token_ngram'])

for i, news_ngram in enumerate(news_ngrams):
    news_ngram = news_ngram.replace("[", "")
    news_ngram = news_ngram.replace("]", "")
    news['token_ngram'][i] = news_ngram
    
news

C:\Users\student\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Unnamed: 0.1,new_content,ngram,new_date,token,token_ngram
0,0,2005년 10년만기 미국 국채수익률이 연방준비제도이사회(FRB)의 금리인상 지속...,"['단기/NNG;국채/NNG;수익률/NNG;상승/NNG', '국채/NNG;수익률/N...",2005-01-01,"['만기/NNG', '국채/NNG', '수익률/NNG', 'fed/NNG', 'fe...","'단기/NNG;국채/NNG;수익률/NNG;상승/NNG', '국채/NNG;수익률/NN..."
1,1,2년만기 국채가격 4년래 최악의 한해 보내 10년만기 미국 국채수익률이 ...,['국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG'],2005-01-01,"['만기/NNG', '국채/NNG', '가격/NNG', '최악/NNG', '보내/V...","'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '만기/NNG..."
2,2,2004년 마지막 거래일인 31일 뉴욕환시에서 미국 달러화는 개장초 102엔 근처...,"['경상/NNG;수지/NNG;적자/NNG', '유로달러/NNG;약세/NNG', '유...",2005-01-01,"['마지막/NNG', '거래일/NNG', '뉴욕/NNG', '환시/NNG', '달러...","'경상/NNG;수지/NNG;적자/NNG', '유로달러/NNG;약세/NNG', '유로..."
3,3,) 한해 마지막 날인 31일 뉴욕 주요 금융시장은 한산한 거래속에 새해를 준비하는 ...,['국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG'],2005-01-01,"['마지막/NNG', '날/NNG', '뉴욕/NNG', '금융시장/NNG', '한산...","'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '마지막/NN..."
4,4,지난해 뉴욕증시는 기술주 주도로 2년 연속연초 대비 상승하면서 대표지수들을 지난 ...,"['국제/NNG;유가/NNG;내림/NNG', '인플레이션/NNG;높/VA', '순이...",2005-01-02,"['주도/NNG', '연속/NNG', '대비/NNG', '상승/NNG', '하/XS...","'국제/NNG;유가/NNG;내림/NNG', '인플레이션/NNG;높/VA', '순이익..."
...,...,...,...,...,...,...
261812,261812,한국은행 11월중 금융기관 가중평균금리주택담보대출도 0.05%p 내린 2.45%로 ...,"['금리/NNG;은행채/NNG;금리/NNG;상승/NNG', '금리/NNG;시장/NN...",2019-12-31,"['중/NNG', '금융기관/NNG', '금리/NNG', '주택담보대출/NNG', ...","'금리/NNG;은행채/NNG;금리/NNG;상승/NNG', '금리/NNG;시장/NNG..."
261813,261813,이주열 한국은행 총재 신년사(사진=연합뉴스)[이데일리 김경은 기자] 이주열(사진) ...,"['물가/NNG;상승률/NNG;낮/VA', '금융시스템/NNG;위험/NNG', '투...",2019-12-31,"['신년사/NNG', '기자/NNG', '가장/MAG', '주력/NNG', '하/V...","'물가/NNG;상승률/NNG;낮/VA', '금융시스템/NNG;위험/NNG', '투자..."
261814,261814,KB증권 보고서[이데일리 전재욱 기자] KB증권은 내년 세계 경제가 미·중 무역분쟁...,"['금리/NNG;경기/NNG;모멘텀/NNG', '경제/NNG;지표/NNG;개선/NN...",2019-12-31,"['보고서/NNG', '기자/NNG', '세계/NNG', '경제/NNG', '미/N...","'금리/NNG;경기/NNG;모멘텀/NNG', '경제/NNG;지표/NNG;개선/NNG..."
261815,261815,은행권서 농협 이대훈·수협 이동빈 행장 대표적카드 '빅3' CEO 모두 쥐띠..삼성...,"['글로벌/NNG;금융시장/NNG;불확실성/NNG;크/VV', '미래/NNG;성장동...",2019-12-31,"['농협/NNG', '수협/NNG', '이동/NNG', '빈/VV', '행장/NNG...","'글로벌/NNG;금융시장/NNG;불확실성/NNG;크/VV', '미래/NNG;성장동력..."


In [5]:
news = news[['new_date', 'token_ngram']]
news

,new_date,token_ngram
0,2005-01-01,"'단기/NNG;국채/NNG;수익률/NNG;상승/NNG', '국채/NNG;수익률/NN..."
1,2005-01-01,"'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '만기/NNG..."
2,2005-01-01,"'경상/NNG;수지/NNG;적자/NNG', '유로달러/NNG;약세/NNG', '유로..."
3,2005-01-01,"'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '마지막/NN..."
4,2005-01-02,"'국제/NNG;유가/NNG;내림/NNG', '인플레이션/NNG;높/VA', '순이익..."
...,...,...
261812,2019-12-31,"'금리/NNG;은행채/NNG;금리/NNG;상승/NNG', '금리/NNG;시장/NNG..."
261813,2019-12-31,"'물가/NNG;상승률/NNG;낮/VA', '금융시스템/NNG;위험/NNG', '투자..."
261814,2019-12-31,"'금리/NNG;경기/NNG;모멘텀/NNG', '경제/NNG;지표/NNG;개선/NNG..."
261815,2019-12-31,"'글로벌/NNG;금융시장/NNG;불확실성/NNG;크/VV', '미래/NNG;성장동력..."


In [6]:
news.rename(columns = {"new_date": "date"}, inplace = True)
# news2 = news1['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) #str형식의 date를 date형식으로 변환
# news2 = news2.sort_values(["date"], ascending=[True])
# news2 = news2.reset_index()
# news2 = news2.drop(0, axis=1)
news

C:\Users\student\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,date,token_ngram
0,2005-01-01,"'단기/NNG;국채/NNG;수익률/NNG;상승/NNG', '국채/NNG;수익률/NN..."
1,2005-01-01,"'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '만기/NNG..."
2,2005-01-01,"'경상/NNG;수지/NNG;적자/NNG', '유로달러/NNG;약세/NNG', '유로..."
3,2005-01-01,"'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '마지막/NN..."
4,2005-01-02,"'국제/NNG;유가/NNG;내림/NNG', '인플레이션/NNG;높/VA', '순이익..."
...,...,...
261812,2019-12-31,"'금리/NNG;은행채/NNG;금리/NNG;상승/NNG', '금리/NNG;시장/NNG..."
261813,2019-12-31,"'물가/NNG;상승률/NNG;낮/VA', '금융시스템/NNG;위험/NNG', '투자..."
261814,2019-12-31,"'금리/NNG;경기/NNG;모멘텀/NNG', '경제/NNG;지표/NNG;개선/NNG..."
261815,2019-12-31,"'글로벌/NNG;금융시장/NNG;불확실성/NNG;크/VV', '미래/NNG;성장동력..."


In [7]:
news1 = news
news1['date'] = news1['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) #str형식의 date를 date형식으로 변환
# news2 = news2.sort_values(["date"], ascending=[True])
# news2 = news2.reset_index()
# news2 = news2.drop(0, axis=1)
news1

C:\Users\student\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,token_ngram
0,2005-01-01,"'단기/NNG;국채/NNG;수익률/NNG;상승/NNG', '국채/NNG;수익률/NN..."
1,2005-01-01,"'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '만기/NNG..."
2,2005-01-01,"'경상/NNG;수지/NNG;적자/NNG', '유로달러/NNG;약세/NNG', '유로..."
3,2005-01-01,"'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '마지막/NN..."
4,2005-01-02,"'국제/NNG;유가/NNG;내림/NNG', '인플레이션/NNG;높/VA', '순이익..."
...,...,...
261812,2019-12-31,"'금리/NNG;은행채/NNG;금리/NNG;상승/NNG', '금리/NNG;시장/NNG..."
261813,2019-12-31,"'물가/NNG;상승률/NNG;낮/VA', '금융시스템/NNG;위험/NNG', '투자..."
261814,2019-12-31,"'금리/NNG;경기/NNG;모멘텀/NNG', '경제/NNG;지표/NNG;개선/NNG..."
261815,2019-12-31,"'글로벌/NNG;금융시장/NNG;불확실성/NNG;크/VV', '미래/NNG;성장동력..."


In [8]:
call_rate.columns = ['date', 'call_rate']
news1 = pd.merge(news, call_rate, how='left')

call_rate.columns = ['before_month', 'call_rate']
news1['before_month'] = news1['date'].apply(lambda x: x - timedelta(days=28)) #한달 전 날짜 생성해서 콜금리와 병합
news1 = pd.merge(news1, call_rate, how = 'left', on='before_month')
news1

,date,token_ngram,call_rate_x,before_month,call_rate_y
0,2005-01-01,"'단기/NNG;국채/NNG;수익률/NNG;상승/NNG', '국채/NNG;수익률/NN...",3.29,2004-12-04,3.26
1,2005-01-01,"'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '만기/NNG...",3.29,2004-12-04,3.26
2,2005-01-01,"'경상/NNG;수지/NNG;적자/NNG', '유로달러/NNG;약세/NNG', '유로...",3.29,2004-12-04,3.26
3,2005-01-01,"'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '마지막/NN...",3.29,2004-12-04,3.26
4,2005-01-02,"'국제/NNG;유가/NNG;내림/NNG', '인플레이션/NNG;높/VA', '순이익...",3.29,2004-12-05,3.26
...,...,...,...,...,...
261812,2019-12-31,"'금리/NNG;은행채/NNG;금리/NNG;상승/NNG', '금리/NNG;시장/NNG...",NaN,2019-12-03,NaN
261813,2019-12-31,"'물가/NNG;상승률/NNG;낮/VA', '금융시스템/NNG;위험/NNG', '투자...",NaN,2019-12-03,NaN
261814,2019-12-31,"'금리/NNG;경기/NNG;모멘텀/NNG', '경제/NNG;지표/NNG;개선/NNG...",NaN,2019-12-03,NaN
261815,2019-12-31,"'글로벌/NNG;금융시장/NNG;불확실성/NNG;크/VV', '미래/NNG;성장동력...",NaN,2019-12-03,NaN


In [9]:
news1 = news1.dropna()
news1['change'] = news1['call_rate_x'] - news1['call_rate_y']
news_label = news1

news1['label'] = news1['change'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0 ) #threshold 3bp를 적용하지 않으면 114개 남음
news = news1[['token_ngram', 'label']]
news

C:\Users\student\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\student\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,token_ngram,label
0,"'단기/NNG;국채/NNG;수익률/NNG;상승/NNG', '국채/NNG;수익률/NN...",1
1,"'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '만기/NNG...",1
2,"'경상/NNG;수지/NNG;적자/NNG', '유로달러/NNG;약세/NNG', '유로...",1
3,"'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '마지막/NN...",1
4,"'국제/NNG;유가/NNG;내림/NNG', '인플레이션/NNG;높/VA', '순이익...",1
...,...,...
258450,"'수출/NNG;경쟁력/NNG;약화/NNG;우려/NNG', '외환시장/NNG;개입/N...",1
258451,"'신규/NNG;취급액/NNG;코픽스/NNG;오르/VV', '금리/NNG;오르/VV;...",1
258452,"'fed/NNG;금리/NNG;인상/NNG', '금리/NNG;인상/NNG;걸림돌/NN...",1
258453,"'은행/NNG;예금/NNG;금리/NNG;오르/VV', '금리/NNG;인상/NNG',...",1


In [10]:
news.to_csv('news_label.csv', encoding='utf-8')

# **6. 채권 분석 보고서**

In [11]:
bond = pd.read_csv('report_original.csv', header=0, index_col=[0])
bond_report = bond[['new_date', 'token_ngram', 'label']]
bond_report.rename(columns = {"new_date": "date"}, inplace = True)
bond_report = bond_report[['token_ngram', 'label']]
bond_report

C:\Users\student\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,token_ngram,label
0,"프레젠테이션/NNG,채권/NNG,채권시장/NNG,전망/NNG,동향/NNG,금리/NN...",1
1,"의사록/NNG,확신/NNG,없/VA,인상/NNG,의견/NNG,있/VA,확인/NNG,...",1
2,"속도조절/NNG,중/NNG,발행시장/NNG,공모/NNG,회사채/NNG,발행/NNG,...",1
3,"fed/NNG,fed/NNG,함의/NNG,fed/NNG,구성/NNG,변화/NNG,채...",1
4,"채권분석/NNG,채권/NNG,fed/NNG,통화정책/NNG,불확실성/NNG,확대/N...",1
...,...,...
1318,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,하락/NNG,하/VV,만기/NNG...",1
1319,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,상승/NNG,하/VV,만기/NNG...",1
1320,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,상승/NNG,하/VV,만기/NNG...",0
1321,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,하락/NNG,하/VV,만기/NNG...",0


In [187]:
bond_report.to_csv('bond_label.csv', encoding='utf-8')

In [12]:
corpus = pd.merge(news, bond_report, how='outer')
corpus = pd.merge(corpus, minutes, how='outer')
corpus

,token_ngram,label
0,"'단기/NNG;국채/NNG;수익률/NNG;상승/NNG', '국채/NNG;수익률/NN...",1
1,"'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '만기/NNG...",1
2,"'경상/NNG;수지/NNG;적자/NNG', '유로달러/NNG;약세/NNG', '유로...",1
3,"'국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG', '마지막/NN...",1
4,"'국제/NNG;유가/NNG;내림/NNG', '인플레이션/NNG;높/VA', '순이익...",1
...,...,...
254491,"'국제/NNG,금융시장/NNG,안정/NNG,되/VV,지속/NNG,하/XSV,가운데/...",0
254492,"'성장/NNG,고용/NNG,회복국면/NNG,정점/NNG,향하/VV,불구/NNG,물가...",0
254493,"'금리정상화/NNG,행보/NNG,불구/NNG,미/NNG,달러/NNG,약세/NNG,기...",1
254494,"'임금/NNG,상승/NNG,확대/NNG,되/XSV,세제/NNG,개혁/NNG,기대/N...",0


In [15]:
corpus.to_csv('corpus_bokyoung.csv', encoding='utf-8')